In [1]:
import os,sys
import pickle
import numpy as np                                       # fast vectors and matrices
import matplotlib.pyplot as plt                          # plotting
from scipy.fftpack import fft

from time import time

sys.path.insert(0,'lib/')
import config
import diagnostics
import base_model

from sklearn.metrics import average_precision_score

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import tensorflow as tfw

%matplotlib inline

with open(config.labels_path, 'rb') as f:
    labels = pickle.load(f)


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# windowed, exponentially distributed filters between 50Hz and 22kHz
def create_filters(d,k):
    x = np.linspace(0, 2*np.pi, d, endpoint=False)
    wsin = np.empty((1,d,1,k), dtype=np.float32)
    wcos = np.empty((1,d,1,k), dtype=np.float32)
    start_freq = 50.
    end_freq = 6000.
    num_cycles = start_freq*d/44000.
    scaling_ind = np.log(end_freq/start_freq)/k
    window_mask = 1.0-1.0*np.cos(x)
    for ind in range(k):
        wsin[0,:,0,ind] = window_mask*np.sin(np.exp(ind*scaling_ind)*num_cycles*x)
        wcos[0,:,0,ind] = window_mask*np.cos(np.exp(ind*scaling_ind)*num_cycles*x)
            
    return wsin,wcos

class Spectrograms(base_model.Model):
    def __init__(self, *args, **kwargs):
        super(Spectrograms, self).__init__(*args, **kwargs)

    def define_graph(self):
        super(Spectrograms, self).define_graph()
        
        # lvl1 convolutions are shared between regions
        self.k = 512              # lvl1 nodes
        self.d = 4096              # lvl1 receptive field
        
        self.k2 = 512              # lvl2 nodes

        # number of lvl1 features
        regions = 1 + (self.window - self.d)/self.stride
        print ('Number of V1 feature regions: {}'.format(regions))

        wsin,wcos = create_filters(self.d,self.k)
        
        wscale = tf.constant(10e-5, dtype=tf.float32)

        with tf.variable_scope('parameters'):
            w1 = tf.Variable(wscale * tf.random_normal([tf.cast(regions, tf.int32) * tf.cast(self.k, tf.int32), self.k2], seed=999))
            w1avg = self.register_weights(w1,'w1',average=.9998)
            beta = tf.Variable(wscale*tf.random_normal([self.k2,self.m],seed=999))
            betaavg = self.register_weights(beta,'beta',average=.9998)

        with tf.variable_scope('queued_model'):
            zx = tf.square(tf.nn.conv2d(self.xq,wsin,strides=[1,1,self.stride,1],padding='VALID')) \
               + tf.square(tf.nn.conv2d(self.xq,wcos,strides=[1,1,self.stride,1],padding='VALID'))
            z2 = tf.nn.relu(tf.matmul(tf.reshape(tf.log(zx + 10e-12),[self.batch_size,tf.cast(regions, tf.int32)*tf.cast(self.k, tf.int32)]),w1))
            y = tf.matmul(z2,beta)
            self.loss = tf.reduce_mean(tf.nn.l2_loss(y-tf.reshape(self.yq,[self.batch_size,self.m])))

        with tf.variable_scope('direct_model'):
            zx = tf.square(tf.nn.conv2d(self.xd,wsin,strides=[1,1,self.stride,1],padding='VALID')) \
               + tf.square(tf.nn.conv2d(self.xd,wcos,strides=[1,1,self.stride,1],padding='VALID'))
            z2 = tf.nn.relu(tf.matmul(tf.reshape(tf.log(zx + 10e-12),[tf.shape(self.xd)[0],tf.cast(regions, tf.int32)*tf.cast(self.k, tf.int32)]),w1avg))
            self.y_direct = tf.matmul(z2,betaavg)
            self.loss_direct = tf.reduce_mean(tf.nn.l2_loss(self.y_direct-self.yd))

In [4]:
model = Spectrograms(labels,checkpoint_path='weights/convnet_mlp_raw/', outputs=1, window=16384, mmap=False, batch_size=100,
                    normalize=True, extended_test_set=True, use_mirex=False, init=False, pitch_transforms=5, jitter=.1,
                    restrict=False)

Number of V1 feature regions: 25.0
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    
Instructions for updating:
To construct input pipelines, use the `tf.data` module.


## Realizando treinamento

In [ ]:
# lr = .000001/3/3
# mom = .95

# # stop the model if it is already running
# model.stop()

# # we have to rebuild the graph every time because input queues can't be reopened
# model.define_graph()

# learning_rate = tf.placeholder(tf.float32, shape=[])
# opt_op = tf.train.MomentumOptimizer(learning_rate,mom).minimize(model.loss)
# with tf.control_dependencies([opt_op]):
#     train_step = tf.group(*model.averages)

# # start up the session, kick off the worker threads, restore checkpoint, etc.
# model.start()

# try:
#     ptime = time()
#     print (model.status_header())
#     while True:
#         if model.iter % 10000 == 0:
#             model.update_status(ptime,time(),lr)
#             model.checkpoint()
#             print (model.status())
#             ptime = time()

#         model.sess.run(train_step, feed_dict={learning_rate: lr})
#         model.iter += 1

# except KeyboardInterrupt:
#     model.checkpoint()
#     print ('Graceful Exit')
# finally:
#     model.stop()

In [ ]:
burnin=100
fig, ((ax1, ax2),(ax3,ax4),(ax5,ax6)) = plt.subplots(3,2)
fig.set_figwidth(12)
fig.set_figheight(10)
ax1.set_title('average precision')
ax1.plot(model.stats['iter'][2][burnin:],model.stats['avp_test'][2][burnin:],color='g')
ax1.plot(model.stats['iter'][2][burnin:],model.stats['avp_train'][2][burnin:],color='b')
ax2.set_title('square loss')
ax2.plot(model.stats['iter'][2][burnin:],model.stats['mse_test'][2][burnin:],color='g')
ax2.plot(model.stats['iter'][2][burnin:],model.stats['mse_train'][2][burnin:],color='b')
ax3.set_title('weights top')
ax3.plot(model.stats['iter'][2][burnin:],model.stats['nbeta'][2][burnin:],color='g')
ax4.set_title('learning rate')
ax4.plot(model.stats['iter'][2][burnin:],model.stats['lr'][2][burnin:],color='g')
ax6.set_title('weights middle')
ax6.plot(model.stats['iter'][2][burnin:],model.stats['nw1'][2][burnin:],color='g')

print (max(model.stats['avp_test'][2]))
print (min(model.stats['mse_test'][2]))

# Mirex Stats

In [ ]:
mse_test, Yhat, Y, mse_breakdown, avp_breakdown = model.sample_records(config.test_ids, 7500, fixed_stride=512)
avp_test = average_precision_score(Y.flatten(),Yhat.flatten())
print (avp_test)

In [6]:
Accavg = Etotavg = 0
for i in range(3):
    _,_,_,Acc,Etot = diagnostics.mirex_statistics(model,i,threshold=.4)
    Accavg += Acc
    Etotavg += Etot

print (Accavg/3.,Etotavg/3.)

f:\TCC\Projeto\lib\base_model.py:168: RuntimeWarning: invalid value encountered in divide
  if self.normalize: x /= np.linalg.norm(x) + config.epsilon


AvgP	P	R	Acc	ETot	ESub	Emiss	Efa
4.29	2.96	72.88	0.03	24.08	0.06	0.21	23.81


f:\TCC\Projeto\lib\base_model.py:168: RuntimeWarning: invalid value encountered in divide
  if self.normalize: x /= np.linalg.norm(x) + config.epsilon


AvgP	P	R	Acc	ETot	ESub	Emiss	Efa
4.76	2.40	73.60	0.02	30.14	0.07	0.20	29.88


f:\TCC\Projeto\lib\base_model.py:168: RuntimeWarning: invalid value encountered in divide
  if self.normalize: x /= np.linalg.norm(x) + config.epsilon


AvgP	P	R	Acc	ETot	ESub	Emiss	Efa
10.16	3.48	65.15	0.03	18.31	0.11	0.24	17.96
0.029083818829743868 24.17844340226082


# Extended test set stats

In [ ]:
mse_test, Yhat, Y, mse_breakdown, avp_breakdown = model.sample_records(config.test_ids_ext, 7500, fixed_stride=512)
avp_test = average_precision_score(Y.flatten(),Yhat.flatten())
print (avp_test)

In [5]:
Accavg = Etotavg = 0
for i in range(10):
    _,_,_,Acc,Etot = diagnostics.mirex_statistics(model,i,threshold=.4)
    Accavg += Acc
    Etotavg += Etot

print (Accavg/10.,Etotavg/10.)

f:\TCC\Projeto\lib\base_model.py:168: RuntimeWarning: invalid value encountered in divide
  if self.normalize: x /= np.linalg.norm(x) + config.epsilon


AvgP	P	R	Acc	ETot	ESub	Emiss	Efa
4.29	2.96	72.88	0.03	24.08	0.06	0.21	23.81


f:\TCC\Projeto\lib\base_model.py:168: RuntimeWarning: invalid value encountered in divide
  if self.normalize: x /= np.linalg.norm(x) + config.epsilon


AvgP	P	R	Acc	ETot	ESub	Emiss	Efa
4.76	2.40	73.60	0.02	30.14	0.07	0.20	29.88


f:\TCC\Projeto\lib\base_model.py:168: RuntimeWarning: invalid value encountered in divide
  if self.normalize: x /= np.linalg.norm(x) + config.epsilon


...

AvgP	P	R	Acc	ETot	ESub	Emiss	Efa
5.82	3.22	66.01	0.03	20.09	0.08	0.26	19.75


f:\TCC\Projeto\lib\base_model.py:168: RuntimeWarning: invalid value encountered in divide
  if self.normalize: x /= np.linalg.norm(x) + config.epsilon


AvgP	P	R	Acc	ETot	ESub	Emiss	Efa
6.14	3.30	68.55	0.03	20.36	0.06	0.26	20.05


f:\TCC\Projeto\lib\base_model.py:168: RuntimeWarning: invalid value encountered in divide
  if self.normalize: x /= np.linalg.norm(x) + config.epsilon


AvgP	P	R	Acc	ETot	ESub	Emiss	Efa
4.77	3.27	67.41	0.03	20.21	0.07	0.26	19.88
0.026102943541238016 30.279090379198863
